In [1]:
from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime, date, timedelta

In [8]:
df_daily_demand = pd.read_csv("Dados/daily_demand.csv")

In [9]:
df_daily_demand["datehour"] = pd.to_datetime(df_daily_demand["datehour"])

In [10]:
df_daily_demand = df_daily_demand.set_index("datehour").asfreq("H").drop(columns=["date"])

In [11]:
df_daily_demand.head()

,total,isholiday,isnearholiday,isweekend,ma_total,TAVG,TMAX,TMIN,PRCP,SNOW,SNWD
datehour,,,,,,,,,,,
2017-01-01 00:00:00,249.0,False,True,True,382.5,44.0,48.0,40.0,0.0,0.0,0.0
2017-01-01 01:00:00,246.0,False,True,True,382.5,44.0,48.0,40.0,0.0,0.0,0.0
2017-01-01 02:00:00,199.0,False,True,True,382.5,44.0,48.0,40.0,0.0,0.0,0.0
2017-01-01 03:00:00,146.0,False,True,True,382.5,44.0,48.0,40.0,0.0,0.0,0.0
2017-01-01 04:00:00,66.0,False,True,True,382.5,44.0,48.0,40.0,0.0,0.0,0.0


In [13]:
from keras.preprocessing import sequence 
from keras.models import Sequential 
from keras.layers import Dense, Embedding 
from keras.layers import LSTM 

In [14]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
 
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [21]:
n_hours = 7*24
n_features = len(df_daily_demand.columns)

In [22]:
values = df_daily_demand.values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [23]:
# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 1)

reframed.head()

,var1(t-168),var2(t-168),var3(t-168),var4(t-168),var5(t-168),var6(t-168),var7(t-168),var8(t-168),var9(t-168),var10(t-168),...,var2(t),var3(t),var4(t),var5(t),var6(t),var7(t),var8(t),var9(t),var10(t),var11(t)
168,0.034124,0.0,1.0,1.0,0.119518,0.440252,0.421687,0.475,0.0,0.0,...,0.0,0.0,1.0,0.059139,0.144654,0.144578,0.175,0.0,0.0,0.493671
169,0.033712,0.0,1.0,1.0,0.119518,0.440252,0.421687,0.475,0.0,0.0,...,0.0,0.0,1.0,0.048974,0.144654,0.144578,0.175,0.0,0.0,0.493671
170,0.027271,0.0,1.0,1.0,0.119518,0.440252,0.421687,0.475,0.0,0.0,...,0.0,0.0,1.0,0.042433,0.144654,0.144578,0.175,0.0,0.0,0.493671
171,0.020008,0.0,1.0,1.0,0.119518,0.440252,0.421687,0.475,0.0,0.0,...,0.0,0.0,1.0,0.039027,0.144654,0.144578,0.175,0.0,0.0,0.493671
172,0.009045,0.0,1.0,1.0,0.119518,0.440252,0.421687,0.475,0.0,0.0,...,0.0,0.0,1.0,0.035798,0.144654,0.144578,0.175,0.0,0.0,0.493671


In [25]:
# split into train and test sets
values = reframed.values
n_train_hours = 365 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
print(train_X.shape, len(train_X), train_y.shape)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(8760, 1848) 8760 (8760,)
(8760, 168, 11) (8760,) (17352, 168, 11) (17352,)


In [26]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

In [ ]:
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/50
122/122 - 22s - loss: 0.0589 - val_loss: 0.0643 - 22s/epoch - 182ms/step
Epoch 2/50
122/122 - 23s - loss: 0.0524 - val_loss: 0.0882 - 23s/epoch - 185ms/step
Epoch 3/50
122/122 - 24s - loss: 0.0493 - val_loss: 0.0787 - 24s/epoch - 199ms/step
Epoch 4/50
122/122 - 23s - loss: 0.0464 - val_loss: 0.0613 - 23s/epoch - 190ms/step
Epoch 5/50
122/122 - 25s - loss: 0.0410 - val_loss: 0.0601 - 25s/epoch - 202ms/step
Epoch 6/50
122/122 - 25s - loss: 0.0381 - val_loss: 0.0600 - 25s/epoch - 209ms/step
Epoch 7/50
122/122 - 26s - loss: 0.0369 - val_loss: 0.0594 - 26s/epoch - 211ms/step
Epoch 8/50
122/122 - 27s - loss: 0.0374 - val_loss: 0.0636 - 27s/epoch - 222ms/step
Epoch 9/50
122/122 - 33s - loss: 0.0353 - val_loss: 0.0557 - 33s/epoch - 271ms/step
Epoch 10/50


In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, -7:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -7:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)